In [33]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

#EKLENEBİLECEKLER: test, dev spliti ve hiperparametre oynancası, loss-time grafikleri

In [17]:
spec = cv2.imread("specs/class0/1.png", 0)
#plt.imshow(spec, cmap="gray")
spec.max()

242

In [25]:
images = []
labels = []

for i in range(10):
    path = "specs/class" + str(i)
    num = len(os.listdir(path))
    for j in range(num):
        img_path = path + "/" + str(j) + ".png"
        img = cv2.imread(img_path, 0)
        img = img / 255
        images.append(img)
        labels.append(i)

images = np.array(images)
labels = np.array(labels)

print(f"len(images) = {len(images)}")
print(f"len(labels) = {len(labels)}")

len(images) = 8732
len(labels) = 8732


In [26]:
images.shape

(8732, 100, 100)

In [59]:
from sklearn.model_selection import train_test_split

train_size = 0.75
x_train, x_test, y_train, y_test = train_test_split(images, labels, train_size=train_size)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

#x_test, x_dev, y_test, y_dev = train_test_split(x_test, y_test)

x_train = x_train.reshape(6549, 100, 100, 1)
x_test = x_test.reshape(2183, 100, 100, 1)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
#plt.imshow(x_train[0], cmap="gray")

(6549, 100, 100) (6549,) (2183, 100, 100) (2183,)
(6549, 100, 100, 1) (6549,) (2183, 100, 100, 1) (2183,)


In [60]:
from keras.utils import to_categorical

y_cat_train = to_categorical(y_train, 10)
print(y_cat_train.shape, y_cat_train[0])

y_cat_test = to_categorical(y_test, 10)

(6549, 10) [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [61]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras.losses import categorical_crossentropy
import warnings
warnings.filterwarnings("ignore")

In [66]:
img_shape = (100, 100, 1)
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=img_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=img_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss=categorical_crossentropy, metrics=['accuracy'])


In [67]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_14 (Conv2D)          (None, 97, 97, 32)        544       
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 48, 48, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_15 (Conv2D)          (None, 45, 45, 32)        16416     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 22, 22, 32)        0         
 ng2D)                                                           
                                                                 
 flatten_6 (Flatten)         (None, 15488)             0         
                                                                 
 dense_12 (Dense)            (None, 256)              

In [94]:
model.fit(x_train, y_cat_train, epochs=5, verbose=1)

#Önce 5, sonra 7 denendi.

Epoch 1/2
205/205 [==============================] - 10s 50ms/step - loss: 0.0665 - accuracy: 0.9818
Epoch 2/2
205/205 [==============================] - 11s 52ms/step - loss: 0.0458 - accuracy: 0.9875


In [95]:
model.metrics_names

['loss', 'accuracy']

In [96]:
model.evaluate(x_test, y_cat_test)

# İsabetlilik %85, meeeh. Daha çalışsa daha yapar bu aslan.

69/69 [==============================] - 1s 13ms/step - loss: 0.5923 - accuracy: 0.8745


[0.5923261046409607, 0.874484658241272]

In [99]:
predictions = model.predict(x_test)
predictions

69/69 [==============================] - 1s 12ms/step


array([[6.3565010e-16, 1.3185196e-10, 3.5958569e-03, ..., 2.1928512e-08,
        2.6085537e-08, 9.9640417e-01],
       [6.0884305e-35, 1.0000000e+00, 4.9076602e-23, ..., 4.7207475e-16,
        7.0858737e-23, 2.2863266e-15],
       [9.9619544e-01, 8.1589373e-05, 2.8890969e-05, ..., 1.3099661e-06,
        7.7356617e-06, 1.0566964e-03],
       ...,
       [3.7863176e-12, 2.4877473e-14, 3.5341738e-08, ..., 7.3961964e-10,
        9.9998868e-01, 2.2815710e-10],
       [2.7985737e-01, 1.7958368e-05, 8.0215953e-02, ..., 4.2712036e-01,
        1.7489624e-03, 1.2234592e-01],
       [0.0000000e+00, 1.1972965e-36, 7.5691267e-24, ..., 1.8488909e-30,
        1.0000000e+00, 4.6722684e-31]], dtype=float32)

In [100]:
print(predictions[0], predictions[0].max())
print(predictions.shape)
#Her sınıf için ihtimaller veriyor. argmax yeter.

[6.3565010e-16 1.3185196e-10 3.5958569e-03 5.7637278e-10 1.9311379e-13
 9.2885666e-17 2.5172986e-22 2.1928512e-08 2.6085537e-08 9.9640417e-01] 0.9964042
(2183, 10)


In [101]:
predicted_classes = np.argmax(predictions, axis=1)
predicted_classes

array([9, 1, 0, ..., 8, 7, 8], dtype=int64)

In [102]:
y_test

array([9, 1, 0, ..., 8, 9, 8])

In [103]:
correct = np.sum(predicted_classes == y_test)
print(f"Number of true classifications was {correct}.\nThis makes our accuracy in test {correct/len(y_test)}")

# Neden manuel? NEDEN OLMASIN?

Number of true classifications was 1909.
This makes our accuracy in test 0.8744846541456711
